In [1]:
import os
import timeit
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy.sparse import random
import numpy as np
from scipy.sparse import coo_matrix
from scipy.io import mmread

#from PE import PE # cycle counting

from time import perf_counter

In [2]:
def RowPartitioning(M,peSize):
    row,column = M.shape
    count = 0
    matrixList = []
    for i in range(row):
        if(count == 0):
            part = np.vstack(M[i:i+1])
            count += 1
            if(i == row - 1):
                matrixList.append(part)
        elif(count < peSize):
            part = np.vstack((part,M[i:i+1]))
            count += 1
            if(i == row - 1):
                matrixList.append(part)
        elif(count == peSize):
            count = 1
            matrixList.append(part)
            part = np.vstack(M[i:i+1])
            if(i == row - 1):
                matrixList.append(part)
    return matrixList

In [3]:
def columnSplit(M,label,peSize):
    r,c = M.shape
    level = []
    levelLabel = []
    x = int(c/peSize)
    for i in range(x):
        subMatrix = M[:,i*peSize:peSize*(i+1)]
        subLabel = label[:,i*peSize:peSize*(i+1)]
        level.append(subMatrix)
        levelLabel.append(subLabel)
    return level,levelLabel

In [4]:
def delColumns(M, label):
        l = []
        r, c = M.shape
        for i in range(c):
            zero = 1
            for j in range(r):
                if (M[..., i][j] != 0):
                    zero = 0
                    break;
            if (zero == 0):
                l.append(i)
        M_list = []
        flag_M_list = []
        for i in range(len(l)):
            M_list.append(M[..., l[i]])
            flag_M_list.append(label[..., l[i]])
        delM = np.array(M_list).T
        delLabel = np.array(flag_M_list).T
        return delM, delLabel

In [5]:
def Mapping(delPackedCombinedMatrixList, delPackedCombinedLabelList):
    mapeddelPackedCombinedLabelList = []
    mapeddelPackedCombinedMatrixList = []
    for i in range(len(delPackedCombinedMatrixList)):
        mapeddelPackedCombinedLabelList.append([])
        mapeddelPackedCombinedMatrixList.append([])
    for i in range(len(delPackedCombinedMatrixList)):
        r, c = delPackedCombinedMatrixList[i].shape
        res = c % peSize[1]
        if (res != 0):
            res = peSize[1] - res
            l, label = [], []
            for j in range(r):
                l.append(0)
                label.append(-1)
            l = np.array(l)
            l = l.reshape(-1, 1)
            label = np.array(label)
            label = label.reshape(-1, 1)
            for j in range(res):
                delPackedCombinedMatrixList[i] = np.hstack((delPackedCombinedMatrixList[i], l))
                delPackedCombinedLabelList[i] = np.hstack((delPackedCombinedLabelList[i], label))
            mapMatrices, mapLabels = columnSplit(delPackedCombinedMatrixList[i], delPackedCombinedLabelList[i],
                                                 peSize[1])
            for item in mapMatrices:
                mapeddelPackedCombinedMatrixList[i].append(item)
            for item in mapLabels:
                mapeddelPackedCombinedLabelList[i].append(item)
        else:
            mapMatrices, mapLabels = columnSplit(delPackedCombinedMatrixList[i], delPackedCombinedLabelList[i],
                                                 peSize[1])
            for item in mapMatrices:
                mapeddelPackedCombinedMatrixList[i].append(item)
            for item in mapLabels:
                mapeddelPackedCombinedLabelList[i].append(item)
    return mapeddelPackedCombinedMatrixList, mapeddelPackedCombinedLabelList

In [6]:
# print("Start time Output: ", StartTime) #start time


stationary = random(40 , 40,density=0.2, random_state=None, data_rvs=None) #change here
stationary = stationary.toarray()
#print(stationary)
streaming = random(40 , 40, density=0.1, random_state=None, data_rvs=None) #change here
streaming = streaming.toarray()

#nnz=int(0.2*(40*40))

In [7]:
peSize = (4,4) #change here

In [8]:
StartTime = perf_counter()

##reordering step
#import numpy as np
#import numpy
#T = np.count_nonzero(stationary[:],axis=-1)
#x=np.argsort(-T) 
#y= np.arange(1000)

#new_order = x    # maps previous index to new index
#new_order2 = y
#stationary = [[stationary[i][j] for j in new_order2] for i in new_order]
#stationary = numpy.array(stationary)
#streaming = [[streaming[i][j] for j in new_order] for i in new_order2]
#streaming = numpy.array(streaming)


import numpy
fin_labelstreaming=[]
def packstationary(M,flag_M):
    r,c=M.shape
    G=[]
    for i in range(c):
        G.append(i)
    d={}
    G = d.fromkeys(G)
    for key in G:
        count=0
        for j in range(r):
            if(M[j][int(key)]!=0):
                count+=1
        G[key]=count
    G = sorted(G.items(), key=lambda d:d[1], reverse = True)
    M_sort=[]
    flag_M_sort=[]
    for item in G:
        column,_=item
        M_sort.append(M[...,column])
        flag_M_sort.append(flag_M[...,column])
    M_sort=np.array(M_sort).T
    flag_M_sort=np.array(flag_M_sort).T
    M_packed=np.zeros((r,c))
    flag_M_packed=np.zeros((r,c))
    for i in range(r):
        l=M_sort[i,...].tolist()
        flag_l=flag_M_sort[i,...].tolist()
        temp=[]
        flag_temp=[]
        for j in range(c):
            if(l[j]!=0):
                temp.append(l[j])
            if(flag_l[j]!=-1):
                flag_temp.append(flag_l[j])
        for k in range(0,c-len(temp)):
            temp.append(0)
            flag_temp.append(-1)
        M_packed[i,...]=np.array(temp)
        flag_M_packed[i,...]=np.array(flag_temp)
    return M_packed,flag_M_packed
def Hpack(stationary,streamingData):
    r,c = stationary.shape
    label=np.zeros((r,c))
    for i in range(c):
        for j in range(r):
            if(stationary[...,i][j]!=0):
                label[...,i][j]=i
            else:
                label[...,i][j]=-1
    partitionList = RowPartitioning(stationary ,peSize[0])
    partitionLabelList = RowPartitioning(label ,peSize[0])
    #print(stationary)
    #print(partitionLabelList)
    columnSplitedMatrixList = []
    columnSplitedLabelList = []
    for i in range(len(partitionList)):
        level,levelLabel = columnSplit(partitionList[i],partitionLabelList[i],peSize[1])
        columnSplitedMatrixList.append(level)
        columnSplitedLabelList.append(levelLabel)
    #print(columnSplitedLabelList)
    #print("______")
    packedColumnSplitedMatrixList = []
    packedColumnSplitedLabelList = []
    for i in range(len(columnSplitedMatrixList)):
        packedLevelMatrix = []
        packedLevelLabel = []
        for j in range(len(columnSplitedMatrixList[i])):
            packedM,packedLabel = packstationary(columnSplitedMatrixList[i][j],columnSplitedLabelList[i][j])
            packedLevelMatrix.append(packedM)
            packedLevelLabel.append(packedLabel)
        packedColumnSplitedMatrixList.append(packedLevelMatrix)
        packedColumnSplitedLabelList.append(packedLevelLabel)
    #print("++++")
    #print(packedColumnSplitedLabelList) packed column id without merging (without del blanc column)
    packedCombinedMatrixList = []
    packedCombinedLabelList = []
    for i in range(len(packedColumnSplitedMatrixList)):
        for j in range(len(packedColumnSplitedMatrixList[i])):
            if(j == 0):
                combineM = np.column_stack(packedColumnSplitedMatrixList[i][j]).T
                combineLabel = np.column_stack(packedColumnSplitedLabelList[i][j]).T
                

            else:
                combineM = np.column_stack((combineM,packedColumnSplitedMatrixList[i][j]))
                combineLabel = np.column_stack((combineLabel,packedColumnSplitedLabelList[i][j]))
        packedCombinedMatrixList.append(combineM)
        packedCombinedLabelList.append(combineLabel)
        #print("chk")
        #print(combineLabel)
    #print("last")
    #print(packedCombinedLabelList)
    delPackedCombinedMatrixList = []
    delPackedCombinedLabelList = []
    for i in range(len(packedCombinedMatrixList)):
        delM,delLabel = delColumns(packedCombinedMatrixList[i],packedCombinedLabelList[i])
        delPackedCombinedMatrixList.append(delM)
        delPackedCombinedLabelList.append(delLabel)
    mapeddelPackedCombinedMatrixList,mapeddelPackedCombinedLabelList = Mapping(delPackedCombinedMatrixList,delPackedCombinedLabelList)
    streaming = streamingData
    r,c = streaming.shape
    label=np.zeros((r,c))
    for i in range(c):
        for j in range(r):
            if(streaming[...,i][j]!=0):
                label[...,i][j]=j
            else:
                label[...,i][j]=-1
    streamingPartitionList = RowPartitioning(streaming.T,peSize[0])
    streamingPartitionLabelList = RowPartitioning(label.T,peSize[0])
    #print(streamingPartitionLabelList)
    #print(streaming)
    streamingPartitionPackedList = []
    streamingPartitionPackedLabelList = []
    for ii in range(len(streamingPartitionList)):
       itemM22 = RowPartitioning(streamingPartitionList[ii].T,peSize[0])
       itemL22 = RowPartitioning(streamingPartitionLabelList[ii].T,peSize[0]) 
       streamingPartitionPackedList.append(itemM22)
       streamingPartitionPackedLabelList.append(itemL22) 
    #X=np.array(streamingPartitionPackedLabelList1).T
    #print(streamingPartitionPackedLabelList)
    #OKKKKKKKKKKKK
    #print("streamingPartitionLabelList")
    #print(streaming)
    #count= 0
    #streamingPartitionPackedList = []
    #streamingPartitionPackedLabelList = []
    #for item in range(len(streamingPartitionList)):
     #   itemM,itemL = columnSplit(streamingPartitionList[item].T,streamingPartitionLabelList [item].T,peSize[1])
     #   streamingPartitionPackedList.append(itemM)
      #  streamingPartitionPackedLabelList.append(itemL)
        #print("-----")
        #print(streamingPartitionPackedLabelList)
    #print(streamingPartitionPackedLabelList)
    #print("-----")
    #print(streamingPartitionPackedLabelList)
    packedstreamingMatrixList = []
    packedstreamingLabelList = []    

    for i in range(len(streamingPartitionPackedList)):
        packedLevelMatrix1 = []
        packedLevelLabel1 = []
        for j in range(len(streamingPartitionPackedList[i])):
        		itemMpacked,itemLpacked = packstationary(streamingPartitionPackedList[i][j].T,streamingPartitionPackedLabelList[i][j].T)
        		packedLevelMatrix1.append(itemMpacked.T)
        		packedLevelLabel1.append(itemLpacked.T)
        packedstreamingMatrixList.append(packedLevelMatrix1)
        packedstreamingLabelList.append(packedLevelLabel1)
        #print("chk")
    #print(packedstreamingLabelList)## OKKKKKKKK (label without merging)
        #fin_label.append(packedstreamingLabelList)

    packedCombinedMatrixList1 = []
    packedCombinedLabelList1 = []
        #count += 1
    for i in range(len(packedstreamingMatrixList)):
         for j in range(len(packedstreamingMatrixList[i])):
            if(j == 0):
                #combineM1 = np.column_stack(packedstreamingMatrixList[i][j]).T
                #combineLabel1 = np.column_stack(packedstreamingLabelList[i][j]).T
                combineM1 = np.column_stack(packedstreamingMatrixList[i][j])
                combineLabel1 = np.column_stack(packedstreamingLabelList[i][j])
            else:
                combineM1 = np.column_stack((combineM1,packedstreamingMatrixList[i][j].T))
                combineLabel1 = np.column_stack((combineLabel1,packedstreamingLabelList[i][j].T))

         packedCombinedMatrixList1.append(combineM1)
         packedCombinedLabelList1.append(combineLabel1)
         #print("chkkkkkk point")
         #print(packedCombinedLabelList1)
    delPackedCombinedMatrixList1 = []
    delPackedCombinedLabelList1 = []
    for ii in range(len(packedCombinedMatrixList1)):
         delM1,delLabel1 = delColumns(packedCombinedMatrixList1[ii],packedCombinedLabelList1[ii])
         delPackedCombinedMatrixList1.append(delM1)
         delPackedCombinedLabelList1.append(delLabel1)
         #print("chkkkkkk 222222")
         #print(delPackedCombinedLabelList1)
    mapeddelPackedstreamingCombinedMatrixList,mapeddelPackedstreamingCombinedLabelList = Mapping(delPackedCombinedMatrixList1,delPackedCombinedLabelList1)
    streaminglabel1=[]
    for dd in range(len(mapeddelPackedstreamingCombinedLabelList)):
       hh=[]
       for gg in range(len(mapeddelPackedstreamingCombinedLabelList[dd][:])):
          hh.append(mapeddelPackedstreamingCombinedLabelList[dd][gg].T)
       streaminglabel1.append(hh)
    fin_labelstreaming.append(streaminglabel1)
    #print(streaminglabel1)
    #mapeddelPackedstreamingCombinedLabelList= streaminglabel1
    #print(mapeddelPackedstreamingCombinedLabelList)
    return mapeddelPackedCombinedMatrixList,mapeddelPackedCombinedLabelList,mapeddelPackedstreamingCombinedMatrixList,fin_labelstreaming,
mapeddelPackedCombinedMatrixList,mapeddelPackedCombinedLabelList,mapeddelPackedstreamingCombinedMatrixList,fin_labelstreaming = Hpack(stationary,streaming)


ENDTIME_H = perf_counter()
print("H_Output Total Time:", ENDTIME_H-StartTime)


H_Output Total Time: 0.023219024000013633


In [11]:
#streaming labels (row id)
fin_labelstreaming[0]
mapeddelPackedCombinedLabelList

[[[array([[ 3., -1., -1., -1.],
          [-1.,  7., -1., -1.],
          [-1., -1.,  9., 11.],
          [-1., 12., -1., 12.]]),
   array([[-1., 22., -1., -1.],
          [27., -1., -1., -1.],
          [31., -1., 28., -1.],
          [-1., 32., 33., -1.]]),
   array([[-1., -1., 35., -1.],
          [36., -1., 36., -1.],
          [38., -1., -1., -1.],
          [-1., -1., -1., -1.]])],
  [array([[-1.,  6., -1., -1.],
          [17., 17., -1., 18.],
          [18., 19., -1., -1.],
          [20., -1., 22., -1.]]),
   array([[-1., 26., 26., -1.],
          [-1., 35., -1., -1.],
          [-1., -1., -1., 37.],
          [-1., -1., -1., -1.]])],
  [array([[ 4.,  6., -1., -1.],
          [-1., -1., -1., 10.],
          [-1., -1., 14., 14.],
          [19., -1., -1., -1.]]),
   array([[-1., 20., -1., -1.],
          [-1., -1., 29., -1.],
          [32., -1., -1., 32.],
          [-1., -1., -1., 36.]])],
  [array([[-1.,  0., -1.,  1.],
          [-1., -1., -1.,  3.],
          [ 7., -1., -1

In [18]:
print(len(fin_labelstreaming[0]))
for i in range(len(fin_labelstreaming[0])):
    print(len(fin_labelstreaming[0][i]))
streamingListFinal = []
for i in range(len(fin_labelstreaming[0])):
    result = None
    for j in range(len(fin_labelstreaming[0][i])):
        if j == 0:
            result = fin_labelstreaming[0][i][j]
        else:
            result = np.vstack([result, fin_labelstreaming[0][i][j]])
    streamingListFinal.append(result)

10
3
2
2
4
3
3
3
3
4
2


In [20]:
print(len(streamingListFinal))
for item in streamingListFinal:
    print(item.shape)

10
(12, 4)
(8, 4)
(8, 4)
(16, 4)
(12, 4)
(12, 4)
(12, 4)
(12, 4)
(16, 4)
(8, 4)


In [22]:
for i in range(len(streamingListFinal)):
    np.savetxt("./data/propose/streaming/streaming" + str(i), streamingListFinal[i], fmt="%d")

In [15]:
#stationary labels (col id)
# mapeddelPackedCombinedLabelList
print(len(mapeddelPackedCombinedLabelList))
for i in range(len(mapeddelPackedCombinedLabelList)):
    print(len(mapeddelPackedCombinedLabelList[i]))

10
5
3
5
4
4
4
4
4
4
4


In [23]:
def mkdir(path):
	folder = os.path.exists(path)
	if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
		os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径
# file = "G:\\xxoo\\test"
# mkdir(file)        
path = "./data/propose/stationary/"
level = len(mapeddelPackedCombinedLabelList)
for i in range(level):
	mkdir(path + "level"+str(i))
	for j in range(len(mapeddelPackedCombinedLabelList[i])):
		np.savetxt(path + "level" + str(i) + "/" + "stationary" + str(j), mapeddelPackedCombinedLabelList[i][j], fmt="%d")